# SLEW Scenario 2

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import os
import subprocess
import urllib.request

#%matplotlib widget

dpsim_path = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')

path_exec = dpsim_path + '/build/Examples/Cxx/'

print(dpsim_path)

name='SP_SynGenTrStab_SMIB_Fault_SlewCase2_JsonSyngenParams'
name_underdamped_case = 'SP_SynGenTrStab_SMIB_Fault_JsonSyngenParams_Underdamped'
name_critically_damped_case = 'SP_SynGenTrStab_SMIB_Fault_JsonSyngenParams_CriticallyDamped'

path_json_underdamped = dpsim_path + '/Configs/example_configs_json/SLEW_Case2_Underdamped.json'
path_json_critically_damped = dpsim_path + '/Configs/example_configs_json/SLEW_Case2_CriticallyDamped.json'

## Run Simulation of Underdamped Case

In [ ]:
sim = subprocess.Popen([path_exec+name, '--params='+path_json_underdamped], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
print(sim.communicate()[0].decode())

## Read Results

In [ ]:
work_dir = 'logs/' + name_underdamped_case + '_SP/'
ts_dpsim_underdamped = read_timeseries_dpsim(work_dir + name_underdamped_case + '_SP.csv')
work_dir

## Generator 1&2 Rotor angle $\delta _r$

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Rotor angle', fontsize=20)
plt.xlabel('time (s)', fontsize=20)
plt.ylabel('angle (deg)', fontsize=20)
plt.plot(ts_dpsim_underdamped['delta_r_gen'].interpolate(50e-6).time, (180/3.14)*ts_dpsim_underdamped['delta_r_gen'].interpolate(50e-6).values, label='$\delta_{r}$')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16,8))
plt.title('electric power', fontsize=20)
plt.xlabel('time (s)', fontsize=20)
plt.ylabel('power (W)', fontsize=20)
plt.plot(ts_dpsim_underdamped['P_elec'].interpolate(50e-6).time, ts_dpsim_underdamped['P_elec'].interpolate(50e-6).values, label='P_elec')
plt.legend()
plt.show()

## Validate Results

In [ ]:
if not os.path.exists('reference-results'):
    os.mkdir('reference-results')

url = 'https://git.rwth-aachen.de/acs/public/simulation/reference-results/raw/master/DPsim/SLEWCases/Case2/SP_SynGenTrStab_SMIB_Fault_SlewCase2_Damping_0.0031825_SP.csv'
local_file = 'reference-results/SP_SynGenTrStab_SMIB_Fault_SlewCase2_UnderDamping_REF.csv'
urllib.request.urlretrieve(url, local_file) 

ts_dpsim_underdamped_REF = read_timeseries_dpsim(local_file)

In [ ]:
p_elec_diff_underdamped = ts_dpsim_underdamped_REF['P_elec'].rmse(ts_dpsim_underdamped['P_elec'], ts_dpsim_underdamped_REF['P_elec'])
delta_diff_underdamped = ts_dpsim_underdamped_REF['delta_r_gen'].rmse(ts_dpsim_underdamped['delta_r_gen'], ts_dpsim_underdamped_REF['delta_r_gen'])
print("Diff P_elec underdamped: " + str(p_elec_diff_underdamped))
assert(p_elec_diff_underdamped==0)
print("Diff Delta_r_gen underdamped: " + str(delta_diff_underdamped))
assert(delta_diff_underdamped==0)

## Run Simulation of Critically Damped Case

In [ ]:
sim = subprocess.Popen([path_exec+name, '--params='+path_json_critically_damped], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
print(sim.communicate()[0].decode())

## Read Results

In [ ]:
work_dir = 'logs/' + name_critically_damped_case + '_SP/'
ts_dpsim_critically_damped = read_timeseries_dpsim(work_dir + name_critically_damped_case + '_SP.csv')
work_dir

## Plot Generator Rotor angle

In [ ]:
plt.figure(figsize=(16,8))
plt.title('Rotor angle', fontsize=20)
plt.xlabel('time (s)', fontsize=20)
plt.ylabel('angle (deg)', fontsize=20)
plt.plot(ts_dpsim_critically_damped['delta_r_gen'].time, (180/3.14)*ts_dpsim_critically_damped['delta_r_gen'].values, label='$\delta_{r}$')
plt.legend()
plt.show()

## Plot Generator Electric Output Power

In [ ]:
plt.figure(figsize=(16,8))
plt.title('electric power', fontsize=20)
plt.xlabel('time (s)', fontsize=20)
plt.ylabel('power (W)', fontsize=20)
plt.plot(ts_dpsim_critically_damped['P_elec'].time, ts_dpsim_critically_damped['P_elec'].values, label='P_elec')
plt.legend()
plt.show()

## Validate Results

In [ ]:
if not os.path.exists('reference-results'):
    os.mkdir('reference-results')

url = 'https://git.rwth-aachen.de/acs/public/simulation/reference-results/raw/master/DPsim/SLEWCases/Case2/SP_SynGenTrStab_SMIB_Fault_SlewCase2_Damping_0.44972_SP.csv'
local_file = 'reference-results/SP_SynGenTrStab_SMIB_Fault_SlewCase2_CriticalDamping_REF.csv'
urllib.request.urlretrieve(url, local_file) 

ts_dpsim_critically_damped_REF = read_timeseries_dpsim(local_file)

In [ ]:
p_elec_diff_critically_damped = ts_dpsim_critically_damped_REF['P_elec'].rmse(ts_dpsim_critically_damped['P_elec'], ts_dpsim_critically_damped_REF['P_elec'])
delta_diff_critically_damped = ts_dpsim_critically_damped_REF['delta_r_gen'].rmse(ts_dpsim_critically_damped['delta_r_gen'], ts_dpsim_critically_damped_REF['delta_r_gen'])
print("Diff P_elec critically damped: " + str(p_elec_diff_critically_damped))
assert(p_elec_diff_critically_damped==0)
print("Diff Delta_r_gen critically damped: " + str(delta_diff_critically_damped))
assert(delta_diff_critically_damped==0)